In [49]:
import pandas as pd
from setfit import AbsaModel, AbsaTrainer, TrainingArguments

from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_dataset
from transformers import EarlyStoppingCallback

from config.config import *

In [17]:
setfit = pd.read_pickle(os.path.join(OBTAINED_DATA, 'setfit.pkl'))
setfit.head()

,text,sentence,span,label,ordinal
0,The solution given here is fantastic and a bit...,The solution given here is fantastic and a bit...,book,positive,0
1,Ahh. I wish I had never gotten this stupid key...,This thought races through Arthur's mind as he...,Arthur,neutral,0
2,Ahh. I wish I had never gotten this stupid key...,This thought races through Arthur's mind as he...,Mister Monday,neutral,0
3,Ahh. I wish I had never gotten this stupid key...,Arthur is put through a rigorous test of brave...,Arthur,neutral,1
4,Ahh. I wish I had never gotten this stupid key...,Arthur is put through a rigorous test of brave...,Mister Monday,negative,1


In [19]:
setfit['label'].value_counts()

label
neutral     2164
positive     956
negative     384
Name: count, dtype: int64

In [20]:
setfit['span'].value_counts()

span
book                 258
Arthur               202
Mister Monday        104
key                   90
Garth Nix             71
                    ... 
clock sword            1
explanation            1
English romances       1
Grown ups              1
fortuitous events      1
Name: count, Length: 1145, dtype: int64

In [22]:
setfit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3504 entries, 0 to 3503
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      3504 non-null   object
 1   sentence  3504 non-null   object
 2   span      3504 non-null   object
 3   label     3504 non-null   object
 4   ordinal   3504 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 137.0+ KB


## Add df to HF

In [31]:
train_df, temp_df = train_test_split(setfit, test_size=0.2, stratify=setfit['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

In [34]:
train_df.to_csv(os.path.join(OBTAINED_DATA, 'hf/setfit_books_reviews_train.csv'), index=False)
val_df.to_csv(os.path.join(OBTAINED_DATA, 'hf/setfit_books_reviews_val.csv'), index=False)
test_df.to_csv(os.path.join(OBTAINED_DATA, 'hf/setfit_books_reviews_test.csv'), index=False)

In [37]:
train_dataset = Dataset.from_csv(os.path.join(OBTAINED_DATA, 'hf/setfit_books_reviews_train.csv'))
val_dataset = Dataset.from_csv(os.path.join(OBTAINED_DATA, 'hf/setfit_books_reviews_val.csv'))
test_dataset = Dataset.from_csv(os.path.join(OBTAINED_DATA, 'hf/setfit_books_reviews_test.csv'))

# Create a DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [39]:
dataset_dict.push_to_hub('omymble/amazon-books-reviews-absa')


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/omymble/amazon-books-reviews-absa/commit/aa5f843230c47eaec05521263e246b1273339562', commit_message='Upload dataset', commit_description='', oid='aa5f843230c47eaec05521263e246b1273339562', pr_url=None, pr_revision=None, pr_num=None)

## Pre-trained model

In [24]:
# Download from the 🤗 Hub
model = AbsaModel.from_pretrained(
    "joshuasundance/setfit-absa-all-MiniLM-L6-v2-laptops-aspect",
    "joshuasundance/setfit-absa-all-mpnet-base-v2-laptops-polarity",
    spacy_model="en_core_web_sm",
)

/Users/arina/GitProjects/amazon_corpus_documents_retrieval/.venv/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.4.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [25]:
preds = model("This laptop meets every expectation and Windows 7 is great!")
preds

[{'span': 'Windows', 'polarity': 'positive'}]

In [26]:
preds = model("Arthur is put through a rigorous test of bravery and compassion to face the evil Mister Monday")
preds

[{'span': 'test', 'polarity': 'neutral'},
 {'span': 'bravery', 'polarity': 'neutral'},
 {'span': 'compassion', 'polarity': 'neutral'}]

In [8]:


# Download from the 🤗 Hub
model = AbsaModel.from_pretrained(
    "joshuasundance/setfit-absa-all-MiniLM-L6-v2-laptops-aspect",
    "joshuasundance/setfit-absa-all-mpnet-base-v2-laptops-polarity",
    spacy_model="en_core_web_sm",
)
# Run inference
preds = model("This laptop meets every expectation and Windows 7 is great!")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3504 entries, 0 to 3503
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      3504 non-null   object
 1   sentence  3504 non-null   object
 2   span      3504 non-null   object
 3   label     3504 non-null   object
 4   ordinal   3504 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 137.0+ KB


/Users/arina/GitProjects/amazon_corpus_documents_retrieval/.venv/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.4.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
preds

[{'span': 'Windows', 'polarity': 'positive'}]

preds1 = model("")
preds1

## example from blog

In [28]:
model = AbsaModel.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/all-mpnet-base-v2",
    spacy_model="en_core_web_sm",
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [42]:
dataset = load_dataset('omymble/amazon-books-reviews-absa')
train_dataset = dataset['train']
val_dataset = dataset['validation']
test_dataset = dataset['test']

Generating train split:   0%|          | 0/2803 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/350 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/351 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="models",
    num_epochs=5,
    use_amp=True,
    batch_size=128,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
)

trainer = AbsaTrainer(
    model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)
trainer.train()

The ordinal of 1 for span 'Mister Monday' in 'Mister Monday is an interesting Fantasy novel that draws readers in from the very beginning. The main character, Arthur is flawed and that is what makes him the "chosen" one for the Keys to the Kingdom. In this series starter, Arthur learns what power he has as he tries to determine how to live his life with this key that he has obtained and this house and little, scary fantasy creatures that no one else can see but him. Arthur starts to think he is losing it, but a few friends from his P. E. class see the creatures too, so Arthur apprehensively goes on his journey as the reluctant hero who is bound to find a way to save his community and family. This book along with Grim Tuesday will hook readers on this series. I can\'t wait for the rest of the books in this series. Harry Potter lovers can definitely find a new love in this series' is too high. Skipping this sample.
The ordinal of 2 for span 'Mister Monday' in "Garth Nix's 'Mister Monday'

Map:   0%|          | 0/11148 [00:00<?, ? examples/s]

Map:   0%|          | 0/2710 [00:00<?, ? examples/s]

In [ ]:
metrics = trainer.evaluate(val_dataset)
print(metrics)

In [ ]:
model.save_pretrained(
    "models/setfit-absa-model-aspect",
    "models/setfit-absa-model-polarity",
)
# or
model.push_to_hub(
    "tomaarsen/setfit-absa-bge-small-en-v1.5-restaurants-aspect",
    "tomaarsen/setfit-absa-bge-small-en-v1.5-restaurants-polarity",
)
